In [ ]:
import h5py
import numpy as np
from glob import glob 
import matplotlib.pyplot as plt
import os

def datx2py(file_name):
    """Loads a .datx into Python, credit goes to gkaplan.
    https://gist.github.com/g-s-k/ccffb1e84df065a690e554f4b40cfd3a"""
    def _group2dict(obj):
        return {k: _decode_h5(v) for k, v in zip(obj.keys(), obj.values())}
    def _struct2dict(obj):
        names = obj.dtype.names
        return [dict(zip(names, _decode_h5(record))) for record in obj]
    def _decode_h5(obj):
        if isinstance(obj, h5py.Group):
            d = _group2dict(obj)
            if len(obj.attrs):
                d['attrs'] = _decode_h5(obj.attrs)
            return d
        elif isinstance(obj, h5py.AttributeManager):
            return _group2dict(obj)
        elif isinstance(obj, h5py.Dataset):
            d = {'attrs': _decode_h5(obj.attrs)}
            try:
                d['vals'] = obj[()]
            except (OSError, TypeError):
                pass
            return d
        elif isinstance(obj, np.ndarray):
            if np.issubdtype(obj.dtype, np.number) and obj.shape == (1,):
                return obj[0]
            elif obj.dtype == 'object':
                return _decode_h5([_decode_h5(o) for o in obj])
            elif np.issubdtype(obj.dtype, np.void):
                return _decode_h5(_struct2dict(obj))
            else:
                return obj
        elif isinstance(obj, np.void):
            return _decode_h5([_decode_h5(o) for o in obj])
        elif isinstance(obj, bytes):
            return obj.decode()
        elif isinstance(obj, list) or isinstance(obj, tuple):
            if len(obj) == 1:
                return obj[0]
            else:
                return obj
        else:
            return obj
    with h5py.File(file_name, 'r') as f:
        h5data = _decode_h5(f)
    return h5data

def get_data(datx_file):
    """Returns the Surface and Intensity data from a single .datx file"""
    myh5 = datx2py(datx_file)                      # File is the string with the location of the file
    zsurf = myh5['Data']['Surface']           # Get the surfaces
    zdata = list(zsurf.values())[0]           # Good for fixing stuff later  
    zsurf = zdata['vals']                     # Get the data from the surface group
    zsurf[zsurf == zdata['attrs']['No Data']] = np.nan  # Write no data as NaNs for compatibility
    zint = myh5['Data']['Intensity']          # Get the intensity group
    zint = list(zint.values())[0]['vals'].astype(float)  # Get the data from the intensity grou[]
    zint[zint>200000] = np.nan                # This fixes the regions left out from stitching
    return zsurf, zint
    
def getcleansurf(img):
    suma = np.nansum(img, axis=0)
    counts = np.sum(~np.isnan(img), axis=0)
    counts[counts<20] = 0
    f = suma/counts
    f[np.isinf(f)] = np.nan
    return f

def find_edge(y, v):
    g2 = np.abs(np.gradient(y))
    idx = np.argwhere(g2>v)
    l = idx[0][0]
    h = idx[-1][0]
    return l,h

def subline(f, offx, offy):
    xchange = -len(f)
    ychange = (f[0]-offx)-(f[-1]-offy)
    xarray = np.arange(len(f))
    yarray = xarray*(ychange/xchange) + (f[0]-offx)
    res = (f-yarray)
    return res

def plot_bounds(i, C, l):
    plt.figure(figsize=(7, 5))
    plt.subplot(211)
    plt.plot(first_pass[i])
    plt.axvline(C[i,0], color="black", linestyle="dashed")
    plt.axvline(C[i,1], color="black", linestyle="dashed")
    plt.subplot(223)
    plt.plot(first_pass[i])
    plt.axvline(C[i,0], color="black", linestyle="dashed")
    plt.axvline(C[i,1], color="black", linestyle="dashed")
    plt.xlim(C[i,0]-l, C[i,0]+l)
    plt.subplot(224)
    plt.plot(first_pass[i])
    plt.axvline(C[i,0], color="black", linestyle="dashed")
    plt.axvline(C[i,1], color="black", linestyle="dashed")
    plt.xlim(C[i,1]-l, C[i,1]+l)
    plt.tight_layout()
    
def plotrel_profile(i, C, o):
    plt.figure(figsize=(14, 5))
    s_past = subline(first_pass[i-1][C[i-1,0]:C[i-1,1]]/1e3, o[i-1,0], o[i-1,1])
    s_curr = subline(first_pass[i][C[i,0]:C[i,1]]/1e3, o[i,0], o[i,1])
    s_futu = subline(first_pass[i+1][C[i+1,0]:C[i+1,1]]/1e3, o[i+1,0], o[i+1,1])

    plt.plot(s_past, label="Past")
    plt.plot(s_curr, label="Current")
    plt.plot(s_futu, label="Future")
    plt.axhline(np.nanmax(s_curr), color="black", linestyle="--")
    plt.axhline(0, color="black", linestyle="--")
    plt.legend()

def shift(n_s, f):
    start[n_s:] += f*np.arange(n-n_s)

# Clean array from multiple .datx

This is slow and takes some user corrections, but it has to get done. 

The Interferometer tends to fail when the slope is *locally* too large, if we pair that with our FOV being 1000x200, full vertical lines of `NaN` can be common (specially in the edges of the biofilm!). This leads the stitching process (from the ZYGO software) to act weird sometimes, and producing errors. 

In order to generate a clean dataset, we use a two step process:
1. Find the biofilm-LB edges on both sides (if edge is out of the FOV is not all lost!)
2. Substract a plane from edge-to-edge. Discuss: earlier times may require a poly2.
3. Correct profiles that aren't consistent. By looking at *previous* and *future* timepoints, we have boundaries for where the *current* one should be. This involves some tweaking of the offsets on LHS and RHS.

We are allowed to do this for two reasons:
1. Temporal resolution is *very good* there is very little growth between $t$ and $t+1$.
2. Analysis on surface fluctuations is **not affected** by these corrections, since we use only the homeland. We have tested also multiple substraction algorithms and they aren't much different from the raw profiles.

So if it doesn't matter on the surface analysis, why should we do it? Because it makes data clearer and clean! It allows for all the other analysis down the pipeline be able to run as short scripts. Clean data is also easier to share, and it makes **pretty plots**.

Simple fixes that do not require justification are the numbered sections 1-2-3. More information on each of these procedures is given on its corresponding section.

In [ ]:

# Load TIMELAPSE files and get times

#folder_tag = "2021-08-27_jt305"
#folder_tag = "2021-09-03_pyeast"
#folder_tag = "2021-06-25_bgt127"
#folder_tag = "2021-07-30_bacillus"
#folder_tag = "2021-10-28_petiteyeastlong"
#folder_tag = "2022-01-28_aerobicyeast"
folder_tag = "2022-02-04_pa01"

folder = "/run/media/pablo/T7/Documents/Research/Biofilms/Data/Interferometry/radial_timelapses/"+folder_tag+"/"
files = glob(folder+"Raw/*.datx")                                           # Folder with all the .datx
files.sort()
n_replicates = 3                                                            # Number of timelapse replicates (numbered)
n_controls = 3
n = int(len(glob(folder+"Raw/*[0-9][0-9][0-9].datx"))/n_replicates)         # Timepoints for each replicate
files_control = files[-n_controls:]                                                 # These are ALL the un-numbered files
idx = np.arange(n)*n_replicates    # Timepoints x replicates                # Total timelapse measurements
files_split = [[] for i in range(n_replicates)]                             
for i in idx:
    for j in range(n_replicates):
        files_split[j].append(files[i+j])

tstart = 1644000200.0
times_tl = np.zeros([n, n_replicates])                                          # Array of all timelapse relative time measurements (hours)
times_cl = np.zeros(len(files_control))                                # Array of all control relative time measurements (hours)
for i in range(n_replicates):
    for j in range(n):
        times_tl[j,i] = (os.path.getmtime(files_split[i][j])-tstart)/3600
for i in range(len(files_control)):
    times_cl[i] = (os.path.getmtime(files_control[i])-tstart)/3600

offnames = [folder+"Clean/offsets_A.npy", folder+"Clean/offsets_B.npy", folder+"Clean/offsets_C.npy"]
Cnames = [folder+"Clean/bounds_A.npy", folder+"Clean/bounds_B.npy", folder+"Clean/bounds_C.npy"]
dnames = [folder+"Clean/displacement_A.npy", folder+"Clean/displacement_B.npy", folder+"Clean/displacement_C.npy"]

C = np.zeros([n, 2]).astype(int)
offsets = np.zeros([n, 2]).astype(float)
print(times_tl[0,0])
np.save(folder+"Clean/times.npy", times_tl)
np.save(folder+"Clean/times_control.npy", times_cl)

# If running all the controls
#C = np.zeros([len(files_control), 2]).astype(int)
#offsets = np.zeros([len(files_control), 2])


## 1. Biofilm edges

In this step we find the left `l` and right `r` edges in the biofilm profile, we use the function `find_edge(profile)`, which will automatically find them for us. Since experimental data is subject to the Zygo stitching algorithm, the algorithm *may* fail, adjusting the edge location **should** be done before the other steps. 
`l` and `r` and then stored in an Array containing all the edge locations `C`.
For this we first lower the dimensionality of all the timelapse `j` into a list of profiles called `first_pass`. That way when we call them individually and plot against each other they are ready to plot.

Then, we loop over the indivdual profiles `i` while also plotting the previous `i-1` and future `i+1` timesteps. We start from a base offset of 0, that should output the relative height of the profiles in relation to the 

In [ ]:
# Load the timelapse here with j
j = 0                                           # Which of the replicates we'll be working on
first_pass = []                                 # We'll do a naive detection of the edges
for i in range(n):
    s = getcleansurf(get_data(files_split[j][i])[0])# Load profiles as a 1-D array
    #s = getcleansurf(get_data(files_control[i])[0]) # Load control profiles as a 1-D array
    first_pass.append(s)
    lim_edge = 10
    if i>30:
        lim_edge = 20
    if i>90:
        lim_edge=40
    l, r = find_edge(s,lim_edge)                      # Here, 15 is the detection threshold
    C[i] = np.array([l,r])
    print(i, end="\r")

In [ ]:
plt.plot(C[30:50,:])

In [ ]:
# Here we loop over each profile
i = 83
#C[i,0] = C[i-1,0]-100
C[i,1] = C[i-1,1]+00

plot_bounds(i, C, 200)

In [ ]:
np.save(folder+"Clean/bounds_A.npy", C)


## 2. Leveling the profiles

This is the slowest part, since it involves looking at 3 consecutive profiles, and correcting for height displacements. For *most* of the timelapse, you should aim to have the <span style="color:#FF7F0E">**current**</span> profile between the <span style="color:#1F77B4">**past** </span> and <span style="color:#2CA02C">**future**</span> ones.
While some particular timepoints are off just because of small miscalculations in finding the edges, more prevalent issues may arise when the actual biofilm-lb border goes out of focus in either the in or out of plane direction. This will lead to a few consecutive offsets in the same side of the sample.


In [ ]:
#5
i = 47
#offsets[143,1] = 
offsets[i,0] += 0.0
offsets[i,1] += 0.0
plotrel_profile(i, C, offsets)
#plt.ylim(0, 700)

In [ ]:
offsets

In [ ]:
np.save(folder+"Clean/offsets_A.npy", offsets)
np.save(folder+"Clean/bounds_A.npy", C)


## 3. Displacement alignment

Now we have a set of profile heights that correspond to the same timelapse. Since we work using the center of the colonies, it would be useful to have everything aligned, so we can get the data quickly.

Of course, the size of the first inoculum is much less than the final point. For this, we center everything into a 2D array with ALL the timelapse profiles.

In [ ]:
# For timelapses
j = 0                                          # Which of the replicates we'll be working on
offsets = np.load(offnames[j])
C = np.load(Cnames[j]).astype(int)
data = []                                 # We'll do a naive detection of the edges
for i in range(n):
    s = getcleansurf(get_data(files_split[j][i])[0])# Load profiles as a 1-D array
    s_curr = subline(s[C[i,0]:C[i,1]]/1e3, offsets[i,0], offsets[i,1]) # Use edges and offsets to subtract
    data.append(s_curr)

In [ ]:
# For controls
#offsets = np.load(folder+"/Clean/offsets_controls.npy")
C = np.load(folder+"/Clean/bounds_control.npy").astype(int)
data = []                                 # We'll do a naive detection of the edges
for i in range(n_controls):
    s = getcleansurf(get_data(files_control[i])[0])# Load profiles as a 1-D array
    s_curr = subline(s[C[i,0]:C[i,1]]/1e3, offsets[i,0], offsets[i,1]) # Use edges and offsets to subtract
    data.append(s_curr)

In [ ]:
S = np.array([len(x) for x in data])
np.max(S)

In [ ]:
L = 34000
IMG = np.zeros([len(data), L])
IMG[:] = np.nan

start = (L-S)/2
start = start.astype(int)

for i in range(len(data)):
    IMG[i, start[i]:len(data[i])+start[i]] = data[i]
plt.figure(figsize=(8,8))
plt.imshow(IMG, aspect=50, clim=(0, 200), origin="lower", cmap="turbo", interpolation="none")

In [ ]:
plt.figure()
plt.plot(np.transpose(IMG));

In [ ]:
np.save(folder+"Clean/bounds_control.npy", C)
np.save(folder+"Clean/offsets_control.npy", offsets)
np.save(folder+"Clean/profiles_control.npy", IMG)
np.save(folder+"Clean/displacement_control.npy", start)

## 4. Save everything into a .csv

Now we have all the required building blocks for our .csv, for each of [A, B, C, control]:
1. Profiles
2. Boundaries
3. Offset
4. Displacement

Now we load everything into a simple panda dataframe, and export it as .csv!

In [ ]:
import pandas as pd
df = pd.DataFrame({'file':[], 'replicate':[], 'time':[], 'border_l':[], 'border_r':[], 'offset_l':[], 'offest_r':[], 'displacement':[]})

replicate = ["A", "B", "C"]
times = np.load(folder+"Clean/times.npy")
# Loop over ABC
for j in range(3):
    offsets = np.load(offnames[j])
    C = np.load(Cnames[j])
    displ = np.load(dnames[j])
    t = times[:,j]
    repl = np.repeat(replicate[j], len(t))
    fnames = [name[len(folder)+4:] for name in files_split[j]]
    b_l, b_r = C[:,0], C[:,1]
    o_l, o_r = offsets[:,0], offsets[:,1]
    tf = pd.DataFrame({'file':fnames, 'replicate':repl, 'time':t, 'border_l':b_l, 'border_r':b_r, 'offset_l':o_l, 'offest_r':o_r, 'displacement':displ})
    df = df.append(tf)
offsets = np.load(folder+"Clean/offsets_control.npy")
C = np.load(folder+"Clean/bounds_control.npy")
displ = np.load(folder+"Clean/displacement_control.npy")
t = np.load(folder+"Clean/times_control.npy")
repl = np.repeat(replicate[j], len(t))
fnames = [name[len(folder)+4:] for name in files_control]
b_l, b_r = C[:,0], C[:,1]
o_l, o_r = offsets[:,0], offsets[:,1]
tf = pd.DataFrame({'file':fnames, 'replicate':repl, 'time':t, 'border_l':b_l, 'border_r':b_r, 'offset_l':o_l, 'offest_r':o_r, 'displacement':displ})
df = df.append(tf)
df.to_csv(folder+"Clean/cleaning.csv", index=False) 
